In [0]:
%md
# Topic

In [0]:
%%sh
# Unified data
aws s3 ls "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.topic.v1/fact/"

In [0]:
%%sh
# Delta lake data
aws s3 ls "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/"

In [0]:

import datetime
from collections import defaultdict

historical_data_s3_path = "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.topic.v1/fact/"
migrated_data_s3_path =  "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/"
historical_topic_df = spark.read.parquet(historical_data_s3_path)
migrated_topic_df = spark.read.format('delta').load(migrated_data_s3_path)

start_date = datetime.date(2019, 7, 1)
end_date = datetime.date(2019, 7, 2)
start_date_str = start_date.strftime("%Y-%m-%d")
end_date_str = end_date.strftime("%Y-%m-%d")
date_range = end_date - start_date
date_range = date_range.days + 1  # int

# main
for _ in range(date_range):
    process_date_str = start_date.strftime("%Y-%m-%d")
    # process_date = start_date

    historical_count = historical_topic_df.where("process_date='{}'".format(process_date_str)).distinct().count()
    migrated_count = migrated_topic_df.where("process_date='{}'".format(process_date_str)).distinct().count()

    if not historical_count == migrated_count:
        print "historical_count = {}, migrated_count = {}".format(historical_count, migrated_count)
    
    start_date += datetime.timedelta(days=1)
    
print "SUCCESS!"


In [0]:

def sample_test(num, process_date_str, process_date, migrated_topic_df, historical_topic_df):
    column_list = migrated_topic_df.columns
    # ['_identifier', 'country_code', 'date', 'device_code', 'language', 'process_date', 'product_id', 'product_version', 'rating', 'review_id', 'topic_ids']

    migrated_50_rows = migrated_topic_df.where("process_date='{}'".format(process_date_str)).head(num) # Row list

    fails_id = defaultdict(list)
    
    for i in range(num):
        review_id = migrated_50_rows[i]["review_id"]
        historical_row = historical_topic_df.where("process_date='{}' and review_id='{}'".format(process_date, review_id)).collect()
        # print historical_row
        for attr in column_list:
            if attr == "process_date":
                continue
            if not migrated_50_rows[i][attr] == historical_row[0][attr]:
                print "FAILED"
                fails_id["process_date={}, review_id={}".format(process_date_str, review_id)].append(attr)
    
    return fails_id

# process_date = start_date
# process_date_str = start_date.strftime("%Y-%m-%d")
# res = sample_test(2, process_date_str, process_date, migrated_topic_df, historical_topic_df)
# print res

start_date = datetime.date(2019, 7, 1)
end_date = datetime.date(2020, 2, 11)
start_date_str = start_date.strftime("%Y-%m-%d")
end_date_str = end_date.strftime("%Y-%m-%d")
date_range = end_date - start_date
date_range = date_range.days + 1  # int

for _ in range(date_range):
    process_date = start_date
    process_date_str = start_date.strftime("%Y-%m-%d")
    
    res = sample_test(10, process_date_str, process_date, migrated_topic_df, historical_topic_df)
    
    start_date += datetime.timedelta(days=1)
    
print "SUCCESS!"

In [0]:

process_date = "2019-07-01"
num = 2

print migrated_topic_df.columns
print historical_topic_df.columns

migrated_50_rows = migrated_topic_df.where("process_date='{}'".format(process_date)).head(num) # Row list
# migrated_reivew_id_list = [migrated_50_rows[i]["review_id"] for i in range(num)]
# historical_50_rows = [historical_topic_df.where("process_date='{}' and review_id='{}'".format(process_date, review_id)).collect() for review_id in migrated_reivew_id_list]

print migrated_50_rows
# print migrated_reivew_id_list
# print historical_50_rows

review_id = migrated_50_rows[0]["review_id"]
historical_row = historical_topic_df.where("process_date='{}' and review_id='{}'".format(process_date, review_id)).collect()

print historical_row

In [0]:

# num = 200
def sample_test(num, process_date):
    migrated_column_list = migrated_topic_df.columns
    historical_column_list = historical_topic_df.columns
    # process_date = "2020-02-02"
    migrated_50_rows = migrated_topic_df.where("process_date='{}'".format(process_date)).head(num) # Row list
    historical_50_rows = historical_topic_df.where("process_date='{}'".format(process_date)).head(num)
    
    # print "historical", historical_50_rows[1]
    # print "migrated", migrated_50_rows[1]
    
    for i in range(num):
        for attr in migrated_column_list:
            if attr == "process_date":
                continue
            if not migrated_50_rows[i][attr] == historical_50_rows[i][attr]:
                print "FAILED"
                print "attr = {}".format(attr)

num = 50
start_date = datetime.date(2019, 7, 1)
end_date = datetime.date(2020, 2, 11)
date_range = end_date - start_date
date_range = date_range.days + 1  # int

for _ in range(date_range):
    process_date = start_date.strftime("%Y-%m-%d")
    sample_test(num, process_date)
    
    start_date += datetime.timedelta(days=1)
print "FINISHED"
# print migrated_column_list
# print historical_column_list

# print migrated_topic_df.printSchema()
# migrated_pandas_df = migrated_topic_df.toPandas()
# historical_pandas_df = historical_df.toPandas()
# historical_all_label = []
# migrated_all_label = []
# for label, _ in migrated_pandas_df.items():
#     migrated_all_label.append(label)

# for label, _ in historical_pandas_df.items():
#     historical_all_label.append(label)

# print migrated_all_label
# print historical_all_label

In [0]:
%md
# Term

In [0]:
%%sh
# Unified data
aws s3 ls "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.term.v1/fact/"

In [0]:
%%sh
aws s3 ls "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.term.v1/fact/"

In [0]:

test_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.term.v1/fact/process_date=2020-02-03/")
test_df.show()

In [0]:

test_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.term.v1/fact/process_date=2020-02-03/")
test_df.show()

In [0]:

import datetime

historical_data_s3_path = "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.term.v1/fact/"
migrated_data_s3_path = "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.term.v1/fact/"
historical_df = spark.read.parquet(historical_data_s3_path)
migrated_df = spark.read.format('delta').load(migrated_data_s3_path)

start_date = datetime.date(2019, 1, 1)
end_date = datetime.date(2019, 1, 2)
start_date_str = start_date.strftime("%Y-%m-%d")
end_date_str = end_date.strftime("%Y-%m-%d")
date_range = end_date - start_date
date_range = date_range.days + 1  # int
# date_list = [base_date - datetime.timedelta(days=x) for x in range(numdays)]

for _ in range(date_range):
    process_date = start_date.strftime("%Y-%m-%d")

    historical_count = historical_df.where("process_date='{}'".format(process_date)).distinct().count()
    migrated_count = migrated_df.where("process_date='{}'".format(process_date)).distinct().count()

    if not historical_count == migrated_count:
        print "process_date={}, historical_count = {}, migrated_count = {}".format(process_date, historical_count, migrated_count)
    
    start_date += datetime.timedelta(days=1)
    
print "SUCCESS!"


In [0]:

start_date = datetime.date(2019, 1, 1)
end_date = datetime.date(2020, 2, 11)
start_date_str = start_date.strftime("%Y-%m-%d")
end_date_str = end_date.strftime("%Y-%m-%d")
date_range = end_date - start_date
date_range = date_range.days + 1  # int
# date_list = [base_date - datetime.timedelta(days=x) for x in range(numdays)]

def sample_test(num, process_date_str, process_date, migrated_topic_df, historical_topic_df):
    column_list = migrated_topic_df.columns
    # ['_identifier', 'country_code', 'date', 'device_code', 'language', 'process_date', 'product_id', 'product_version', 'rating', 'review_id', 'topic_ids']

    migrated_50_rows = migrated_topic_df.where("process_date='{}'".format(process_date_str)).head(num) # Row list

    fails_id = defaultdict(list)
    
    for i in range(num):
        review_id = migrated_50_rows[i]["review_id"]
        historical_row = historical_topic_df.where("process_date='{}' and review_id='{}'".format(process_date, review_id)).collect()
        if not historical_row:
            print "Historical_row is empty", historical_row
            return

        for attr in column_list:
            if attr == "process_date":
                continue
            if not migrated_50_rows[i][attr] == historical_row[0][attr]:
                print "FAILED"
                print "review_id=", review_id
                temp = fails_id["process_date={}, review_id={}".format(process_date_str, review_id)]
                temp.append(migrated_50_rows)
                temp.append(historical_row)
                break
    
    return fails_id

# process_date = start_date
# process_date_str = start_date.strftime("%Y-%m-%d")
# res = sample_test(2, process_date_str, process_date, migrated_topic_df, historical_topic_df)
# print res
res = []
for _ in range(date_range):
    process_date = start_date
    process_date_str = start_date.strftime("%Y-%m-%d")
    
    temp = sample_test(1, process_date_str, process_date, migrated_df, historical_df)
    if temp:
        res.append(temp)
    
    start_date += datetime.timedelta(days=1)
    
print "SUCCESS!"


In [0]:

print res
print historical_df.where("review_id=5419093684").collect()

In [0]:

migrated_df.where("review_id=5419093684").collect()

In [0]:

print res
def sample_test(num, process_date_str, process_date, migrated_topic_df, historical_topic_df):
    column_list = migrated_topic_df.columns
    # ['_identifier', 'country_code', 'date', 'device_code', 'language', 'process_date', 'product_id', 'product_version', 'rating', 'review_id', 'topic_ids']

    migrated_50_rows = migrated_topic_df.where("process_date='{}'".format(process_date_str)).head(num) # Row list
    # print migrated_50_rows
    fails_id = defaultdict(dict)
    
    for i in range(num):
        review_id = migrated_50_rows[i]["review_id"]
        historical_row = historical_topic_df.where("process_date='{}' and review_id='{}'".format(process_date, review_id)).collect()
        # historical_row = historical_topic_df.where("review_id='{}'".format(review_id)).collect()
        # print historical_row
        for attr in column_list:
            if attr == "process_date":
                continue
            # print i
            # print historical_row
            if not migrated_50_rows[i][attr] == historical_row[0][attr]:
                print "FAILED"
                print "review_id=", review_id
                temp = fails_id["process_date={}, review_id={}".format(process_date_str, review_id)]
                temp["migrated_50_rows"] = migrated_50_rows
                temp["historical_row"] = historical_row
                break
    
    return fails_id
res2 = sample_test(5, "2020-02-03", datetime.date(2020, 2, 3), migrated_df, historical_df)
print res2

In [0]:

for x, y in res2.items():
    print "date&id:{}  migrated:{}  historical:{}".format(x, y["migrated_50_rows"][0]["_identifier"], y["historical_row"][0]["_identifier"])

In [0]:

num = 50
migrated_column_list = migrated_df.columns
historical_column_list = historical_df.columns
process_date = "2020-02-02"
migrated_50_rows = migrated_df.where("process_date='{}'".format(process_date)).head(num) # Row list
historical_50_rows = historical_df.where("process_date='{}'".format(process_date)).head(num)

# print "historical", historical_50_rows[1]
# print "migrated", migrated_50_rows[1]

for i in range(num):
    for attr in migrated_column_list:
        if attr == "process_date":
            continue
        if not migrated_50_rows[i][attr] == historical_50_rows[i][attr]:
            print "FAILED"
            print "attr = {}".format(attr)
print "SUCCESS"

In [0]:
%%sh
aws s3 ls "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.term.v1/fact/" --recursive --human --summarize | tail -3

In [0]:

migrated_pandas_df = migrated_df.toPandas()
# historical_pandas_df = historical_df.toPandas()
# historical_all_label = []
# migrated_all_label = []
# for label, _ in migrated_pandas_df.items():
#     migrated_all_label.append(label)

# for label, _ in historical_pandas_df.items():
#     historical_all_label.append(label)

# print migrated_all_label
# print historical_all_label

In [0]:

import datetime

a = datetime.date(2019, 1, 3)
b = a - datetime.date(2019, 1, 1)
a = a.strftime("%Y-%m-%d")
# b = b.strftime("%d")
print a.__repr__(), b.days.__repr__()

In [0]:

unified_topic_df = spark.read.format('delta').load("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/")

In [0]:
%%sh
aws s3 ls "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.topic.v1/fact/process_date=2020-02-07/"

In [0]:

from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline.type_ import DataType, EventType, ManipulationType
from aadatapipelinecore.core.fs.driver import unified

granularity_list = ['m','w']
def get_path_list(granularity):
    s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-usage'))
    path = s3_bucket_list.all(prefix="unified/city-level-usage.v1/fact/granularity={}".format(granularity), depth_is_1=True)
    # print path[0].split("=")[-1].strip("/")
    # print [x for x in path[x].split("=")[-1].strip("/") ]
    date_list = [(granularity, x.split("=")[-1].strip("/")) for x in path ]
    return date_list
### here you may need to update it with spark.read.format("delta").......instead of unified.read.
def get_delta_lake_version(granularity, date):
    urn = Urn(
            namespace="aa.city-level-usage.v2",
            event="transform",
            data_type=DataType.FACT
        )
    df_v2 = unified.read(spark, urn,  sql_where="granularity='{}' and date='{}'".format(granularity, date)).data_dto_list[0].dataframe
    v2_data_count = df_v2.count()
    return v2_data_count
def get_old_version(granularity, date):
    df_v1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-usage/unified/city-level-usage.v1/fact/granularity={}/date={}/".format(granularity, date))
    return df_v1.count()
def compare(granularity, date):
    count1 = get_delta_lake_version(granularity, date)
    count2 = get_old_version(granularity, date)
    if count1 != count2:
        print 'Not equal!!!!! v1 : {}, v2 : {}, granularity: {}, date: {}'.format(count1, count2, granularity, date)
    else:
        print 'v1 : {}, v2 : {}, granularity: {}, date: {}'.format(count1, count2, granularity, date)
test_path = []
for g in granularity_list:
    test_path.extend(get_path_list(g))
for case in test_path:
    print case
    compare(case[0],case[1])


In [0]:
%angular
<h1>DB Check<h1>

In [0]:
%md
## Unified Review With Topic

In [0]:

from bdce.common.utils import update_application_code
update_application_code(
    spark, role="BDP-PROD-APP-INT-QA", application_name="zidong-topic-join-review"
)
# reload dependencies from temp
# spark.sparkContext.addPyFile("/tmp/zeppelin_application_code/libs/python/dependencies.zip")


In [0]:
%%sh
aws s3 ls "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/"


In [0]:

# Copyright (c) 2019 App Annie Inc. All rights reserved.

"""
Advanced Review Database Check:
    1. The number of data in "unified layer" is equal to elasticsearch
"""

from collections import defaultdict
from applications.db_check_v1.common.base_test import PipelineTest
from applications.db_check_v1.common.table_common_info import urn
from aadatapipelinecore.core.loader import es


class AdvancedReviewUnifiedData(object):
    """
    Get data from unified s3 bucket, then return a DataFrame
    """
    _unified_s3_path = "s3://b2c-prod-data-pipeline-unified-advancedreview/" \
                       "unified/advancedreview.topic.v1/fact/process_date={}"

    def __init__(self, spark):
        self.spark = spark

    def get(self, date):
        """
        :param date: process_date in s3 bucket path
        :type date: string
        :return: unified_df
        :rtype: DataFrame
        """
        unified_df = self.spark.read.parquet(self._unified_s3_path.format(date))
        return unified_df


class AdvancedReviewDBData(object):
    """
    Get data from database (ElasticSearch)
    """
    _common_config = {"database": "adv_review"}
    _index_config = "int-ss-advancedreview_v1*"

    def get(self, country_code, device_code, identifier):
        """
        :return: elasticsearch result
        :rtype: dict
        """
        query_body = {
            "track_total_hits": "true",
            "query": {
                "bool": {
                    "must": [
                        {"match": {"country_code": country_code}},
                        {"match": {"device_code": device_code}},
                        {"match": {"_identifier": identifier}}
                    ]
                }
            }
        }
        es_conn = es.connection(urn, self._common_config)
        es_data = es_conn.search(
            index=self._index_config,
            body=query_body)
        return es_data


class TestTopicJoinReview(PipelineTest):
    """
    Compare ES data amount with unified review data
    """
    trigger_date_config = ("10 10 * * *", 2)

    def setUp(self):
        self.country_code_list = ["US", "GB", "CA", "AU", "IN", "JP", "KR"]
        self.device_code_list = ["ios-all", "android-all"]
        self.granularity = 'daily'
        self.failed_ids = defaultdict(list)

    def _compare_diff(self, device_code, country_code, unified_df, identifier):
        unified_count = unified_df.where(
            "country_code='{}' AND device_code='{}'".format(country_code, device_code)).count()
        es_data = AdvancedReviewDBData().get(country_code, device_code, identifier)
        es_total = es_data["hits"]["total"]["value"]
        if not es_total == unified_count:
            self.failed_ids[device_code].append(country_code)

    def test_advanced_review_topic_join_review(self):
        unified_df = AdvancedReviewUnifiedData(self.spark).get(self.check_date_str)
        identifier = unified_df.collect()[0]["_identifier"]
        for device_code in self.device_code_list:
            for country_code in self.country_code_list:
                self._compare_diff(device_code, country_code, unified_df, identifier)

        self.assertTrue(len(self.failed_ids) == 0, self.failed_ids)




In [0]:

# Copyright (c) 2019 App Annie Inc. All rights reserved.

"""
Advanced Review Database Check:
    1. The number of data in "unified layer" is equal to elasticsearch
"""
import unittest
from collections import defaultdict
from applications.db_check_v1.common.base_test import PipelineTest
from applications.db_check_v1.common.db_check_utils import etl_skip
from applications.db_check_v1.common.table_common_info import urn
from aadatapipelinecore.core.loader import es
from applications.db_check_v1.common.db_check_utils import _get_date_from_refresh_routing_config


class AdvancedReviewUnifiedData(object):
    """
    Get data from unified s3 bucket
    """
    _unified_s3_path = "s3://b2c-prod-data-pipeline-unified-advancedreview/" \
                      "unified/advancedreview.topic.v1/fact/process_date={}"

    def __init__(self, spark):
        self.spark = spark

    def get(self, date):
        unified_df = self.spark.read.parquet(self._unified_s3_path.format(date))
        return unified_df


class AdvancedReviewDBData(object):
    _common_config = {"database": "adv_review"}
    _index_config = "int-ss-advancedreview_v1*"

    def __init__(self, country, device, identifier):
        self.country_code = country
        self.device_code = device
        self.identifier = identifier

    def get(self):
        """
        :return: elasticsearch result
        """
        query_body = {
            "track_total_hits": "true",
            "query": {
                "bool": {
                    "must": [
                        {"match": {"country_code": self.country_code}},
                        {"match": {"device_code": self.device_code}},
                        {"match": {"_identifier": self.identifier}}
                    ]
                }
            }
        }
        es_conn = es.connection(urn, self._common_config)
        es_data = es_conn.search(
            index=self._index_config,
            body=query_body)
        return es_data


class TestTopicJoinReview(PipelineTest):
    """
    Compare ES data amount with unified review data
    """
    # Only check today's data
    trigger_date_config = ("0 4 * * *", 1)
    check_date = _get_date_from_refresh_routing_config(trigger_date_config)

    def setUp(self):
        self.country_code_list = ["US", "GB", "CA", "AU", "IN", "JP", "KR"]
        self.device_code_list = ["ios-all", "android-all"]
        self.granularity = 'daily'
        self.unified_df = AdvancedReviewUnifiedData(self.spark).get(self.check_date)
        self.identifier = self.unified_df.collect()[0]["_identifier"]
        self.failed_ids = defaultdict(list)

    def _compare_diff(self, device_code, country_code, unified_df):
        unified_count = unified_df.where(
            "country_code='{}' AND device_code='{}'".format(country_code, device_code)).count()
        print "unified_count", unified_count
        es_data = AdvancedReviewDBData(country_code, device_code, self.identifier).get()
        es_total = es_data["hits"]["total"]["value"]
        if not es_total == unified_count:
            self.failed_ids[device_code].append(country_code)

    def test_topic_join_review(self):
        """
        :return: None
        """
        unified_df = AdvancedReviewUnifiedData(self.spark).get(self.check_date)
        for device_code in self.device_code_list:
            for country_code in self.country_code_list:
                self._compare_diff(device_code, country_code, unified_df)

        self.assertTrue(len(self.failed_ids) == 0, self.failed_ids)


suite = unittest.TestSuite()
suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestTopicJoinReview))


In [0]:

from applications.db_check_v1.common.db_check_utils import _get_date_from_refresh_routing_config

trigger_date_config = ("0 4 * * *", 1)
check_date = _get_date_from_refresh_routing_config(trigger_date_config)
print check_date
unified_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/process_date={}".format(check_date))
print unified_df.distinct().count()

In [0]:
%%sh
aws s3 ls "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/"

In [0]:

type(unified_df)

In [0]:

import unittest
from collections import defaultdict
from applications.db_check_v1.common.base_test import PipelineTest
from applications.db_check_v1.common.db_check_utils import etl_skip
from applications.db_check_v1.common.table_common_info import urn
from aadatapipelinecore.core.loader import es
from applications.db_check_v1.common.db_check_utils import _get_date_from_refresh_routing_config

class AdvancedReviewDBData(object):
    _common_config = {"database": "adv_review"}
    _index_config = "int-ss-advancedreview_v1*"

    def __init__(self, device, identifier):
        self.country_code = country
        self.device_code = device
        self.identifier = identifier

    def get(self):
        """
        :return: elasticsearch result
        """
        query_body = {
            "track_total_hits": "true",
            "query": {
                "bool": {
                    "must": [
                        {"match": {"device_code": self.device_code}},
                        {"match": {"_identifier": self.identifier}}
                    ]
                }
            }
        }
        es_conn = es.connection(urn, self._common_config)
        es_data = es_conn.search(
            index=self._index_config,
            body=query_body)
        return es_data

print AdvancedReviewDBData()

In [0]:

import unittest

class TestNotebook(unittest.TestCase):

    def test_add(self):
        self.assertEqual(add(2, 2), 4)


unittest.main(argv=[''], verbosity=2)

In [0]:

spark = create_spark()
unified_review = spark.read.format("delta").load(
    "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/")
# Only check today's data
trigger_date_config = ("0 4 * * *", 1)
check_date = _get_date_from_refresh_routing_config(trigger_date_config)
data_num_at_process_date = unified_review.where(
    "process_date='{}'".format(check_date)).distinct().count()
# if data_num_at_process_date == 0:
#     trigger_date_config = ("0 4 * * *", 2)
#     check_date = _get_date_from_refresh_routing_config(trigger_date_config)
es_identifier = unified_review.where(
    "process_date='{}'".format(check_date)).collect()[0]._identifier


In [0]:

print check_date, data_num_at_process_date, es_identifier

In [0]:

# Copyright (c) 2019 App Annie Inc. All rights reserved.

"""
Advanced Review Database Check:
    1. The number of data in "unified layer" is equal to elasticsearch
"""

import unittest
from collections import defaultdict
from aadatapipelinecore.core.utils.spark import create_spark
from applications.db_check_v1.common.table_common_info import urn
from aadatapipelinecore.core.loader import es
from applications.db_check_v1.common.db_check_utils import _get_date_from_refresh_routing_config


class TestESdata(unittest.TestCase):
    """
    Compare ES data amount with unified review data
    """
    spark = create_spark()
    unified_review = spark.read.format("delta").load(
        "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/")
    # Only check today's data
    trigger_date_config = ("0 4 * * *", 1)
    check_date = _get_date_from_refresh_routing_config(trigger_date_config)
    data_num_at_process_date = unified_review.where(
        "process_date='{}'".format(check_date)).distinct().count()
    if data_num_at_process_date == 0:
        trigger_date_config = ("0 4 * * *", 2)
        check_date = _get_date_from_refresh_routing_config(trigger_date_config)

    def setUp(self):
        self.country_code_list = ["US", "GB", "CA", "AU", "IN", "JP", "KR"]
        self.device_code_list = ["ios-all", "android-all"]
        self.granularity = 'daily'
        self.es_identifier = self.unified_review.where(
            "process_date='{}'".format(self.check_date)).collect()[0]["_identifier"]

    def test_es_data(self):
        """
        :return: None
        """

        failed_ids = defaultdict(list)

        for device_code in self.device_code_list:
            for country_code in self.country_code_list:
                unified_count = self.unified_review.where(
                    "process_date='{}' AND country_code='{}' AND device_code='{}'".format(
                        self.check_date, country_code, device_code)).count()
                query_body = {
                    "track_total_hits": "true",
                    "query": {
                        "bool": {
                            "must": [
                                {"match": {"country_code": country_code}},
                                {"match": {"device_code": device_code}},
                                {"match": {"_identifier": self.es_identifier}}
                            ]
                        }
                    }
                }
                es_data = self.get_es_db(query_body)
                es_count = es_data["hits"]["total"]["value"]
                if not es_count == unified_count:
                    failed_ids[device_code].append(country_code)

        self.assertTrue(len(failed_ids) == 0, failed_ids)

    def get_es_db(self, query_body):
        """
        :param query_body: elasticsearch query script
        :return: elasticsearch result
        """
        common_config = {"database": "adv_review"}
        es_conn = es.connection(urn, common_config)
        es_data = es_conn.search(
            index="int-ss-advancedreview_v1*",
            body=query_body)
        return es_data



unittest.main(argv=[''], verbosity=2, exit=False)

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/process_date=2020-02-12/

In [0]:
%%sh
aws s3 ls "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/aa.adv_not_join/process_date=2020-02-25/"

In [0]:

try:
    df_test = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/aa.adv_not_join/process_date=2020-02-25/")
except:
    pass
print df_test.collect()


In [0]:
%%sh
aws s3 ls "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.market-size.v1/fact/"


In [0]:
%%sh
# aws s3 ls "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/process_date=2020-02-26/"
aws s3 ls "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/aa.adv_not_join/"

In [0]:

not_join_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/aa.adv_not_join/process_date=2020-03-07/")
not_join_df.count()

In [0]:

unified_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/process_date=2020-02-26").toPandas()
unified_df = unified_df.drop(["_identifier"], axis=1)
print unified_df

In [0]:

df_unified_review.printSchema()

In [0]:
%md
# ADV REVIEW DB CHECK


In [0]:

"""
@param: df_unified_review DataFrame
"""
# adv_review = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.topic.v1/fact/process_date=2019-07-01/")
df_unified_review = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/")
type(df_unified_review)


In [0]:

df_unified_review.where("review_id='5580841379'").show()
# df_unified_review.select(df_unified_review.review_id, df_unified_review._identifier.between(120200225075429899, 120200225075430000)).where("process_date='2020-02-24'").show()

In [0]:

from applications.db_check_v1.common.db_check_utils import _get_date_from_refresh_routing_config
import datetime

# 2 days less than now
# process_date = datetime.datetime.today() - datetime.timedelta(days=2)

# print "process_date", process_date.__repr__()

# print "year=", process_date.year

# Only check today's data
process_date_config = ("10 10 * * *", 1)
# es_identifier_date_config = ("0 4 * * *", 1)
process_date = _get_date_from_refresh_routing_config(process_date_config)
print "Process Date =\t\t", process_date.__repr__()

# Number of data at process_date
date_num_at_process_date = df_unified_review.where("process_date='{}'".format(process_date)).distinct().count()
print "Unified Review Count =\t", date_num_at_process_date.__repr__()


# es_identifier_date = _get_date_from_refresh_routing_config(es_identifier_date_config)
# print "ES Identifier Date =\t", es_identifier_date.__repr__()

es_identifier = df_unified_review.where("process_date='{}'".format(process_date)).collect()[0]._identifier

print "es_identifier =\t\t", es_identifier

# check _identifier
# print df_unified_review.groupby("_identifier").collect().show()

# es_identifier_date_digit = ""
# for i in es_identifier_date:
#     if i.isdigit():
#         es_identifier_date_digit += i
# es_identifier_lower = "1{}000000000".format(es_identifier_date_digit)
# es_identifier_upper = "1{}999999999".format(es_identifier_date_digit)

# print es_identifier_lower.__repr__()
# print es_identifier_upper.__repr__()
# {
#   "track_total_hits": "true", 
#   "query": {
#     "bool": {
#       "must": [
#         {"match": {"country_code": country_code}},
#         {"match": {"device_code": device_code}}
#       ],
#       "filter": {"range": {
#         "_identifier": {
#           "gte": es_identifier_lower,
#           "lte": es_identifier_upper
#         }
#       }}
#     }
#   }
# }

In [0]:

from collections import defaultdict
from applications.db_check_v1.common.table_common_info import urn
from aadatapipelinecore.core.loader import es


class TestESdata():

    def es_data_check(self):
        country_code_list = ["US", "GB", "CA", "AU", "IN", "JP", "KR"]
        device_code_list = ["ios-all", "android-all"]
        failed_es_data_list = []
        failed_list = []
        sum = 0
        
        for device_code in device_code_list:
            for country_code in country_code_list:
                unified_count = df_unified_review.where("process_date='{}' AND country_code='{}' AND device_code='{}'".format(process_date, country_code, device_code)).count()
                query_body = {
                              "track_total_hits": "true", 
                              "query": {
                                "bool": {
                                  "must": [
                                    {"match": {"country_code": country_code}},
                                    {"match": {"device_code": device_code}},
                                    {"match": {"_identifier": es_identifier}}
                                  ]
                                }
                              }
                            }
                res = self.get_es_db(country_code, query_body)
                es_count = res["hits"]["total"]["value"]
                if not es_count == unified_count:
                    failed_list.append((country_code, device_code))
                    failed_es_data_list.append(res)
                sum += es_count
        
        return sum, failed_list, failed_es_data_list

    def get_es_db(self, country_code, query_body):
        common_config = { "database": "adv_review" }
        es_conn = es.connection(urn, common_config)
        res = es_conn.search(
            index="int-ss-advancedreview_v1*",
            body=query_body)
        return res
        

es_total_count, failed_list, es_data = TestESdata().es_data_check()



In [0]:

print es_total_count, failed_list
print len(es_data)
print "diff = ", date_num_at_process_date - es_total_count

# ('US', 'ios-all') ==> es_data[0]
# TODO: loop
review_id_dic = {}
for i in range(len(failed_list)):
    review_id_list = df_unified_review.where("country_code='{}' AND device_code='{}' AND process_date='{}'".format(failed_list[i][0], failed_list[i][1], process_date)).collect()
    review_id_list = [row["review_id"] for row in review_id_list]
    review_id_dic[failed_list[i]] = review_id_list
# print review_id_dic[('GB', 'ios-all')][:10]



In [0]:

from collections import defaultdict
from applications.db_check_v1.common.db_check_utils import _get_date_from_refresh_routing_config
from applications.db_check_v1.common.table_common_info import urn
from aadatapipelinecore.core.loader import es

class FetchData():

    trigger_date_config = ("10 10 * * *", 2)
    check_date = _get_date_from_refresh_routing_config(trigger_date_config)
    print check_date

    granularity = 'daily'

    def get_es_data(self, review_id):
        # review_id = 5571421762
        query_body = {
                      "query": {
                        "bool": {
                          "must": [
                                      { "match": { "country_code": "US" } },
                                      { "match": { "device_code": "ios-all"}}
                            ]
                        }
                      }
                    }
        res = self.connect_es(query_body)
        review_count = len(res["hits"]["hits"])
        if review_count == 1:
            return True
        return "debug: count: {}, res: {}".format(review_count, res)

    def connect_es(self, query_body):
        common_config = { "database": "app_ss" }
        es_conn = es.connection(urn, common_config)
        res = es_conn.search(
            index="int-ss-advancedreview_v1*",
            # index="_all",
            body=query_body)
        return res


es_raw_data = FetchData().get_es_data(5571421762)
es_raw_data

def test():
    review_ids = [row["review_id"] for row in df_unified_review.where("process_date='2020-02-24'").collect()]
    # review_ids = review_ids[:100]
    for review_id in review_ids:
        in_es = FetchData().get_es_data(review_id)
        if not in_es:
            return in_es
    return "Success"

test_res = test()

In [0]:

# Copyright (c) 2019 App Annie Inc. All rights reserved.

"""
Test Advanced Review's tables
"""

import unittest
from collections import defaultdict
from applications.db_check_v1.common.constants import query, citus_settings
from applications.db_check_v1.common.db_check_utils import _get_date_from_refresh_routing_config


class TestAdvancedReview(unittest.TestCase):

    trigger_date_config = ("* * * * *", 2)
    check_date = _get_date_from_refresh_routing_config(trigger_date_config)

    def setUp(self):
        self.table_list = ["topic", "term"]
        self.granularity = 'd'
        self.countries_list = ["US", "GB", "CA", "AU", "IN", "JP", "KR"]

    def check_advanced_review(self):
        for table in self.table_list:
            self.advanced_review_db_check_test(table)

    def advanced_review_db_check_test(self, table):
        device_code = 'ios-all'

        failed_ids = defaultdict(list)
        for country_code in self.countries_list:
            advanced_review_db = self.get_advanced_review_date(table, country_code)
            if not advanced_review_db[0][0]:
                failed_ids[device_code].append(country_code)
        self.assertTrue(len(failed_ids) == 0, failed_ids)

    def get_advanced_review_date(self, table, country_code):
        sql = '''
        select count(1) from advancedreview.advancedreview_{table}_fact_v3 where date = '{date}'
                and country_code = '{country_code}';
        '''.format(table=table, date=self.check_date, country_code=country_code)
        result = query(citus_settings("aa"), sql)
        return result

    def test_advanced_review_daily(self):
        self.check_advanced_review()

unittest.main(argv=[''], verbosity=2)

In [0]:

import unittest
from applications.db_check_v1.cases.advanced_review.test_topic_join_review import TestTopicJoinReview

def suite():
    suite = unittest.TestSuite()
    suite.addTest(TestTopicJoinReview('test_advanced_review_topic_join_review'))
    return suite

if __name__ == '__main__':
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(suite())

In [0]:

import unittest
from applications.db_check_v1.cases.advanced_review.test_topic_join_review import TestTopicJoinReview

def suite():
    suite = unittest.TestSuite()
    suite.addTest(TestTopicJoinReview('test_advanced_review_topic_join_review'))
    return suite

if __name__ == '__main__':
    runner2 = unittest.TextTestRunner(verbosity=2)
    runner2.run(suite())